In [54]:
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax import nnx
import jax.sharding
import tqdm
import jax.numpy as jnp
import time

from etils import ecolab
import treescope
treescope.basic_interactive_setup(autovisualize_arrays=True)

with ecolab.adhoc('xmanager-codelab', reload='parallax'):
  import parallax
  from parallax import ddp
  from parallax.examples import encoder_decoder
  from parallax.examples.utils import print_tpu_memory, clear_tpu_memory
  from parallax import DataParallelTraining
  from parallax import sharding_utils


from google3.perftools.accelerators.xprof.api.colab import xprof

In [55]:
vocab_size = 1000
emb_dim = 128
ff_dim = 1024
num_heads = 8
attn_dim = num_heads * 64
pad_token = 0
num_encoder = 4
num_decoder = 4
enc_seq_len = 80
dec_seq_len = 80
dropout_rate = 0.1

rng = nnx.Rngs(0)
model =  encoder_decoder.TransformerModel(
                        num_encoder=num_encoder,
                        num_decoder=num_decoder,
                        enc_seq_len=enc_seq_len,
                        dec_seq_len=dec_seq_len,
                        vocab_size=vocab_size,
                        embed_dim=emb_dim,
                        feedforward_dim=ff_dim,
                        num_heads=num_heads,
                        dropout_rate=dropout_rate,
                        attn_dim=attn_dim,
                        rngs=rng)

optimizer = nnx.ModelAndOptimizer(model, optax.adamw(learning_rate=5e-5))

In [4]:
print(model)

TransformerModel( # Param: 5,667,304 (22.7 MB), RngState: 2 (12 B), Total: 5,667,306 (22.7 MB)
  pad_token=0,
  enc_seq_len=20,
  dec_seq_len=20,
  vocab_size=1000,
  embed_dim=128,
  feedforward_dim=1024,
  num_heads=8,
  dropout_rate=0.1,
  attn_dim=512,
  encoder=TransformerEncoder( # Param: 2,110,464 (8.4 MB)
    embed_dim=128,
    feedforward_dim=1024,
    num_heads=8,
    num_layers=4,
    attn_dim=512,
    dropout_rate=0.1,
    layers=[TransformerEncoderBlock( # Param: 527,616 (2.1 MB)
      embed_dim=128,
      num_heads=8,
      feedforward_dim=1024,
      dropout_rate=0.1,
      attn_dim=512,
      attention=MultiHeadAttention( # Param: 263,808 (1.1 MB)
        num_heads=8,
        in_features=128,
        qkv_features=512,
        out_features=128,
        in_kv_features=128,
        dtype=None,
        param_dtype=float32,
        broadcast_dropout=True,
        dropout_rate=0.0,
        deterministic=None,
        precision=None,
        kernel_init=<function variance_scal

# Define some utility functions

In [56]:
def generate_reverse_batch(batch_size, seq_len, vocab_size, key):
    input_ids = jax.random.randint(key, (batch_size, seq_len), 1, vocab_size)
    labels = jnp.flip(input_ids, axis=1)

    return {
        "input_ids": input_ids,
        "labels": labels,
    }


In [57]:
def compute_loss(logits, labels):
    loss = optax.softmax_cross_entropy_with_integer_labels(logits=logits, labels=labels)
    return jnp.mean(loss)

@nnx.jit
def train_step(model, optimizer, batch):
    def loss_fn(model, train_encoder_input, train_target_output):
        logits = model(train_encoder_input, train_target_output)
        loss = compute_loss(logits, train_target_output)
        return loss

    grad_fn = nnx.value_and_grad(loss_fn)
    loss, grads = grad_fn(model, jnp.array(batch["input_ids"]), jnp.array(batch["labels"]))
    optimizer.update(grads)
    return loss

@nnx.jit
def eval_step(model, batch, eval_metrics):
    logits = model(jnp.array(batch["input_ids"]), jnp.array(batch["labels"]))
    loss = compute_loss(logits, jnp.array(batch["labels"]))
    labels = jnp.array(batch["labels"])

    eval_metrics.update(
        loss=loss,
        logits=logits,
        labels=labels,
    )

eval_metrics = nnx.MultiMetric(
    loss=nnx.metrics.Average('loss'),
    accuracy=nnx.metrics.Accuracy(),
)

train_metrics_history = {
    "train_loss": [],
}

eval_metrics_history = {
    "test_loss": [],
    "test_accuracy": [],
}



In [62]:
num_train_data = 1680
num_epochs = 10

bar_format = "{desc}[{n_fmt}/{total_fmt}]{postfix} [{elapsed}<{remaining}]"


def train_one_epoch(epoch, batch_size, sharding=False):
    model.train()  # Set model to training mode

    train_total_steps = num_train_data // batch_size  # fixed global steps

    with tqdm.tqdm(
        desc=f"[train] epoch: {epoch}",
        total=train_total_steps,
        bar_format=bar_format,
        leave=True,
    ) as pbar:

        key = jax.random.PRNGKey(epoch)

        for step in range(train_total_steps):
            key, subkey = jax.random.split(key)

            batch = generate_reverse_batch(
                batch_size=batch_size,
                seq_len=enc_seq_len,
                vocab_size=vocab_size,
                key=subkey,
            )

            if sharding:
                batch, _ = sharding_utils.get_sharded_data(batch, None)

            loss = train_step(model, optimizer, batch)
            train_metrics_history["train_loss"].append(loss.item())

            pbar.set_postfix({"loss": loss.item()})
            pbar.update(1)


def evaluate_model(epoch):
    # Compute the metrics on the train and val sets after each training epoch.
    model.eval()  # Set model to evaluation model: e.g. use stored batch statistics

    key = jax.random.PRNGKey(999 + epoch)
    eval_metrics.reset()  # Reset the eval metrics
    val_batch = generate_reverse_batch(
    batch_size=batch_size,
    seq_len=enc_seq_len,
    vocab_size=vocab_size,
    key=key
    )
    eval_step(model, val_batch, eval_metrics)

    for metric, value in eval_metrics.compute().items():
        eval_metrics_history[f'test_{metric}'].append(value)

    print(f"[test] epoch: {epoch + 1}/{num_epochs}")
    print(f"- total loss: {eval_metrics_history['test_loss'][-1]:0.4f}")
    print(f"- Accuracy: {eval_metrics_history['test_accuracy'][-1]:0.4f}")


# Train model without sharding

In [59]:
start_time = time.time()

for epoch in range(num_epochs):
    train_one_epoch(epoch, batch_size=32, sharding=False)
    evaluate_model(epoch)

end_time = time.time()
print(f"Total time: {end_time - start_time:.2f} seconds")

[train] epoch: 0[52/52], loss=6.88 [00:15<00:00]


[test] epoch: 1/10
- total loss: 6.8778
- Accuracy: 0.0037


[train] epoch: 1[52/52], loss=6.47 [00:01<00:00]


[test] epoch: 2/10
- total loss: 6.4660
- Accuracy: 0.0223


[train] epoch: 2[52/52], loss=6 [00:01<00:00]


...

[test] epoch: 8/10
- total loss: 3.8601
- Accuracy: 0.9783


[train] epoch: 8[52/52], loss=3.5 [00:02<00:00]


[test] epoch: 9/10
- total loss: 3.4843
- Accuracy: 0.9881


[train] epoch: 9[52/52], loss=3.12 [00:01<00:00]


[test] epoch: 10/10
- total loss: 3.1156
- Accuracy: 0.9950
Total time: 40.40 seconds


# Train model with DDP

In [60]:
print('Before DDP:')
jax.debug.visualize_array_sharding(model.decoder.layers[0].feedforward.layers[0].kernel.value)

Before DDP:
┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                    TPU 0                                     │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘


In [61]:
ddp = DataParallelTraining(optimizer)
sharded_model, sharded_optimizer = ddp.get_sharded_components()

print("After DDP: ")
jax.debug.visualize_array_sharding(sharded_model.decoder.layers[0].feedforward.layers[0].kernel.value)


INFO:2025-06-10 00:17:23,275:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.


After DDP: 
┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                             TPU 0,1,2,3,4,5,6,7                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘


In [63]:
model = sharded_model
optimizer = sharded_optimizer

start_time = time.time()

for epoch in range(num_epochs):
    train_one_epoch(epoch, batch_size=256, sharding=True)
    evaluate_model(epoch)

end_time = time.time()
print(f"Total time: {end_time - start_time:.2f} seconds")

[train] epoch: 0[0/6] [00:00<?]INFO:2025-06-10 00:17:37,450:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[1/6], loss=3.12 [00:14<01:12]INFO:2025-06-10 00:17:51,990:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[2/6], loss=3.22 [00:14<00:24]INFO:2025-06-10 00:17:52,102:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[3/6], loss=3.12 [00:15<00:10]INFO:2025-06-10 00:17:52,650:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[3/6], loss=3.12 [00:15<00:10]INFO:2025-06-10 00:17:52,702:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[5/6], loss=3.13 [00:15<00:01]INFO:2025-06-10 00:17:52,752:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 0[6/6], loss=3.1

[test] epoch: 1/10
- total loss: 3.0906
- Accuracy: 0.9956


[train] epoch: 1[0/6] [00:00<?]INFO:2025-06-10 00:17:58,351:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[0/6], loss=3.09 [00:00<?]INFO:2025-06-10 00:17:58,407:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[2/6], loss=3.08 [00:00<00:00]INFO:2025-06-10 00:17:58,458:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[2/6], loss=3.07 [00:00<00:00]INFO:2025-06-10 00:17:58,509:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[4/6], loss=3.06 [00:00<00:00]INFO:2025-06-10 00:17:58,560:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[4/6], loss=3.06 [00:00<00:00]INFO:2025-06-10 00:17:58,611:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 1[6/6], loss=3.05 [0

[test] epoch: 2/10
- total loss: 3.0348
- Accuracy: 0.9956


[train] epoch: 2[0/6] [00:00<?]INFO:2025-06-10 00:17:58,734:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[0/6], loss=3.04 [00:00<?]INFO:2025-06-10 00:17:58,785:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[2/6], loss=3.03 [00:00<00:00]INFO:2025-06-10 00:17:58,836:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[2/6], loss=3.01 [00:00<00:00]INFO:2025-06-10 00:17:58,887:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[4/6], loss=3.01 [00:00<00:00]INFO:2025-06-10 00:17:58,939:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[4/6], loss=3 [00:00<00:00]   INFO:2025-06-10 00:17:58,989:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 2[6/6], loss=2.99 [0

...

[test] epoch: 8/10
- total loss: 2.7435
- Accuracy: 0.9978


[train] epoch: 8[0/6] [00:00<?]INFO:2025-06-10 00:18:01,452:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[0/6], loss=2.74 [00:00<?]INFO:2025-06-10 00:18:01,502:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[1/6], loss=2.73 [00:00<00:00]INFO:2025-06-10 00:18:01,551:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[3/6], loss=2.73 [00:00<00:00]INFO:2025-06-10 00:18:01,601:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[3/6], loss=2.71 [00:00<00:00]INFO:2025-06-10 00:18:01,651:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[4/6], loss=2.71 [00:00<00:00]INFO:2025-06-10 00:18:01,701:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 8[6/6], loss=2.7 [00

[test] epoch: 9/10
- total loss: 2.6942
- Accuracy: 0.9980


[train] epoch: 9[0/6] [00:00<?]INFO:2025-06-10 00:18:01,822:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[0/6], loss=2.69 [00:00<?]INFO:2025-06-10 00:18:01,872:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[2/6], loss=2.68 [00:00<00:00]INFO:2025-06-10 00:18:01,923:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[2/6], loss=2.67 [00:00<00:00]INFO:2025-06-10 00:18:01,980:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[4/6], loss=2.67 [00:00<00:00]INFO:2025-06-10 00:18:02,030:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[4/6], loss=2.66 [00:00<00:00]INFO:2025-06-10 00:18:02,079:jax._src.mesh_utils:83: Reordering mesh to physical ring order on single-tray TPU v2/v3.
[train] epoch: 9[6/6], loss=2.65 [0

[test] epoch: 10/10
- total loss: 2.6427
- Accuracy: 0.9982
Total time: 24.76 seconds
